In [ ]:
#=
using Base.Cartesian

const X = @eval @ntuple 3 k->Axis{k}();
=#

In [ ]:
using Revise, ArrayOperations

In [ ]:
struct MyBin <: Primitive{Arity{2}} end

(::MyBin)((x, y)::NTuple{2,AbstractVector}, i::Int) = x[i] * y[i-1]

In [ ]:
n = 2
x, y = rand(n), rand(n)

In [ ]:
# binary operator
f = MyBin()

# fix all arguments
h = f((x, y))

# fix all but first, then fix first and check that result is same
g₁ = Loose{1}(f, (y,))
h′ = g₁((x,))

@assert isequal(h, h′)

# fix all but second, then fix second and check that result is same
g₂ = Loose{2}(f, (x,))
h″ = g₂((y,))

@assert isequal(h, h″)

In [ ]:
# account for non-locality by hand for now

rng = 2:n

@assert isequal(h[rng], x[rng] .* y[rng .- 1])

# Derivatives

In [ ]:
const ∇₁ = ∇{Tuple{1}}()
const ∇₂ = ∇{Tuple{2}}()

const ∂₁ = ∂{1}()
const ∂₂ = ∂{2}()

## First order

In [ ]:
# binary operator
f = MyBin()

d₁f = ∇₁(f)

f₁ = ∂₁(f, (y,))
df₁ = ∇₁(f₁)

@assert isequal(df₁, ∂₁(d₁f, (y,)))

d₂f = ∇₂(f)

f₂ = ∂₂(f, (x,))
df₂ = ∇₁(f₂)

@assert isequal(df₂, ∂₂(d₂f, (x,)))

## Second order

In [ ]:
const ∇₁₂ = ∇{Tuple{1,2}}()

In [ ]:
h = ∇₁₂(f)

# Support

In [ ]:
struct LocalOp <: Primitive{Arity{2}} end

(::LocalOp)((x, y)::NTuple{2,AbstractVector}, i::Int) = x[i] * y[i]

OperatorSupport(::Type{<:LocalOp}, ::Dim) = HasStencil()

Stencil(::Type{<:LocalOp}, ::Dim) = PointWise()

(::Jac{1,S,O})((_, y)::NTuple{2,AbstractVector}, i::Int) where {S,O<:LocalOp} = y[i]
(::Jac{2,S,O})((x, _)::NTuple{2,AbstractVector}, i::Int) where {S,O<:LocalOp} = x[i]

In [ ]:
f = LocalOp()

d₁f = ∇₁(f)
d₂f = ∇₂(f)

@assert isequal(d₁f((x, y), 2), y[2])
@assert isequal(d₂f((x, y), 2), x[2])